In [254]:
import pint
import pyproj
import projnames
import geopandas as gpd
import pandas as pd
import numpy as np
import pint_pandas
ureg = pint.UnitRegistry()

In [255]:
epsgs = list(projnames.by_epsg.keys()) + [4326]

In [256]:
for epsg in epsgs:
    ureg.define('epsg_%s = [epsg_%s]; offset: 1 = _' % (epsg, epsg))

In [257]:
ureg.define('@alias epsg_4326 = latlon')

In [265]:
class C(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __sub__(self, other):
        if not isinstance(other, C): return self
        return C(self.x - other.x, self.y - other.y)
    def __mul__(self, other):
        return C(self.x * other, self.y * other)
    def __truediv__(self, other):
        return C(self.x / other, self.y / other)
    def __add__(self, other):
        if not isinstance(other, C): return self
        return C(self.x + other.x, self.y + other.y)
    def __format__(self, format_spec):
        return str(self.x) + ";" + str(self.y)

In [259]:
def register_epsg(epsg, c):
    def tolatlon(ureg, x):
        return ureg.Quantity(
            C(*pyproj.Transformer.from_crs(
                         epsg, 4326, always_xy=True).transform(x.m.x, x.m.y)), "latlon")
    def fromlatlon(ureg, x):
        return ureg.Quantity(
            C(*pyproj.Transformer.from_crs(
                         4326, epsg, always_xy=True).transform(x.m.x, x.m.y)), "epsg_%s" % epsg)

    c.add_transformation("epsg_%s" % epsg, 'latlon', tolatlon)
    c.add_transformation("latlon", "epsg_%s" % epsg, fromlatlon)

    if (epsg in projnames.by_epsg) and ("UTM" in projnames.by_epsg[epsg]):
        c.add_transformation("delta_epsg_%s" % epsg, 'm',
                             lambda ureg, x: ureg.Quantity((x.m.x**2+x.m.y**2)**0.5, "m"))

c = pint.Context('maps')
for epsg in epsgs:
    register_epsg(epsg, c)

ureg.add_context(c)

In [266]:
p1 = ureg.Quantity(C(15, 2), "epsg_32611")
p2 = ureg.Quantity(C(2, 1), "epsg_32611")
d = p1 - p2

In [267]:
p1.to("latlon", "maps")

<__main__.C at 0x7f65cb1886d0> <Unit('epsg_4326')>

In [268]:
d.to("m", "maps")

13.038404810405298 <Unit('meter')>

In [269]:
p3 = ureg.Quantity(C(72, 10), "latlon")
p4 = ureg.Quantity(C(73, 9), "latlon")

In [270]:
(p4 - p3).to("m", "maps")

9966961.317621892 <Unit('meter')>

In [273]:
df = pd.DataFrame({
     "p": pd.Series([p1, p2], dtype=pint_pandas.pint_array.PintType(p1.u))
})

/home/redhog/Projects/Emerald/emerald-interpret-gui/env/lib/python3.9/site-packages/pint_pandas/pint_array.py:194: RuntimeWarning: pint-pandas does not support magnitudes of <class '__main__.C'>. Converting magnitudes to float.
  warnings.warn(


TypeError: float() argument must be a string or a number, not 'C'

In [271]:
df = pd.DataFrame({
     "torque": pd.Series([1+3j, 2+4j, 2+5j, 3], dtype="pint[lbf ft]"),
     "angular_velocity": pd.Series([1, 2, 2, 3], dtype="pint[rpm]"),
})

/home/redhog/Projects/Emerald/emerald-interpret-gui/env/lib/python3.9/site-packages/pint_pandas/pint_array.py:194: RuntimeWarning: pint-pandas does not support magnitudes of <class 'complex'>. Converting magnitudes to float.
  warnings.warn(


TypeError: can't convert complex to float

In [136]:
ureg.foot * ureg.force_pound

<Unit('foot * force_pound')>

In [ ]:
pint_pandas.pint_array.PintType

In [156]:
a = pd.Series([p1, p2], dtype=pint_pandas.pint_array.PintType(p1.u))

In [192]:
a

AttributeError: 'numpy.ndarray' object has no attribute 'magnitude'

In [193]:
p1.magnitude

array([15,  2])

In [162]:
type(df.torque.values)

pint_pandas.pint_array.PintArray

In [183]:
try:
    print(pd.Series(pint_pandas.pint_array.PintArray([p1, p2], p1.u)))
except:
    import sys, pdb
    sys.last_traceback = sys.exc_info()[2]
    pdb.pm()

> /home/redhog/Projects/Emerald/emerald-interpret-gui/env/lib/python3.9/site-packages/pint_pandas/pint_array.py(286)formatting_function()
    284         def formatting_function(quantity):
    285             return "{:{float_format}}".format(
--> 286                 quantity.magnitude, float_format=float_format
    287             )
    288 

ipdb> quantity
array([15.0, 2.0], dtype=object)
ipdb> bt
  /tmp/ipykernel_2401/3529560940.py(2)<cell line: 1>()
      1 try:
----> 2     print(pd.Series(pint_pandas.pint_array.PintArray([p1, p2], p1.u)))
      3 except:
      4     import sys, pdb
      5     sys.last_traceback = sys.exc_info()[2]

  /home/redhog/Projects/Emerald/emerald-interpret-gui/env/lib/python3.9/site-packages/pandas/core/series.py(1504)__repr__()
   1502         """
   1503         repr_params = fmt.get_series_repr_params()
-> 1504         return self.to_string(**repr_params)
   1505 
   1506     def to_string(

  /home/redhog/Projects/Emerald/emerald-interpret-gui/env/lib

ipdb> list
    281             self.dtype.ureg.default_format
    282         )
    283 
    284         def formatting_function(quantity):
    285             return "{:{float_format}}".format(
--> 286                 quantity.magnitude, float_format=float_format
    287             )
    288 
    289         return formatting_function
    290 
    291     def isna(self):

ipdb> up
> /home/redhog/Projects/Emerald/emerald-interpret-gui/env/lib/python3.9/site-packages/pandas/io/formats/format.py(1390)_format()
   1388             else:
   1389                 # object dtype
-> 1390                 return str(formatter(x))
   1391 
   1392         vals = extract_array(self.values, extract_numpy=True)

ipdb> x
array([15.0, 2.0], dtype=object)
ipdb> formatter
<function PintArray._formatter.<locals>.formatting_function at 0x7f65d0cfaaf0>
ipdb> exit


In [187]:
pint_pandas.pint_array.__file__

'/home/redhog/Projects/Emerald/emerald-interpret-gui/env/lib/python3.9/site-packages/pint_pandas/pint_array.py'

In [176]:
np.array([p1, p2])

/tmp/ipykernel_2401/3639727533.py:1: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  np.array([p1, p2])


array([[15,  2],
       [ 2,  1]])

In [188]:
((4+3j) * ureg.m).magnitude

(4+3j)

In [274]:
type(1+2j)

complex